In [27]:
import pandas as pd
import pickle
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector, RFE, SelectFromModel
import seaborn as sns

In [28]:
df = pd.read_csv("./dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93312 entries, 0 to 93311
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       93312 non-null  float64
 1   Occupation                93312 non-null  int64  
 2   Annual_Income             93312 non-null  float64
 3   Monthly_Inhand_Salary     93312 non-null  float64
 4   Num_Bank_Accounts         93312 non-null  float64
 5   Num_Credit_Card           93312 non-null  float64
 6   Interest_Rate             93312 non-null  float64
 7   Num_of_Loan               93312 non-null  int64  
 8   Delay_from_due_date       93312 non-null  float64
 9   Num_of_Delayed_Payment    93312 non-null  float64
 10  Changed_Credit_Limit      93312 non-null  float64
 11  Num_Credit_Inquiries      93312 non-null  float64
 12  Credit_Mix                93312 non-null  int64  
 13  Outstanding_Debt          93312 non-null  float64
 14  Credit

In [29]:
labels = ['Poor', 'Standard', 'Good']

In [30]:
X = df.drop(['Credit_Score'], axis=1)
y = df['Credit_Score']
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=36)

print(X_train.shape, y_train.shape)

(74649, 30) (74649,)


In [31]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred, labels=np.unique(y_test)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_test, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor              3671              1456           128
True Standard          1496              7453          1131
True Good               131              1110          2087
              precision    recall  f1-score   support

        Poor       0.69      0.70      0.70      5255
    Standard       0.74      0.74      0.74     10080
        Good       0.62      0.63      0.63      3328

    accuracy                           0.71     18663
   macro avg       0.69      0.69      0.69     18663
weighted avg       0.71      0.71      0.71     18663



Độ chính xác khi sử dụng DecisionTree là khoảng 71%, cải thiện 4% so với việc không xử lí dữ liệu rỗng hoặc sai.

Nhãn Good và Poor có f1-score cải thiện rõ rệt, có thể do việc xử lí dữ liệu làm các điểm phân cách giữa Standard và Good/Poor trở nên rõ ràng và chính xác hơn.

In [32]:
y_pred = clf.predict(X_train)
print(pd.DataFrame(metrics.confusion_matrix(y_train, y_pred, labels=np.unique(y_train)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_train, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor             21021                 0             0
True Standard             0             40318             0
True Good                 0                 0         13310
              precision    recall  f1-score   support

        Poor       1.00      1.00      1.00     21021
    Standard       1.00      1.00      1.00     40318
        Good       1.00      1.00      1.00     13310

    accuracy                           1.00     74649
   macro avg       1.00      1.00      1.00     74649
weighted avg       1.00      1.00      1.00     74649



Tuy nhiên, do số lượng feature đầu vào là rất nhiều (30 features), việc sử dụng một Decision Tree đơn giản có thể dễ dàng xảy ra overfit do dễ dàng tạo ra một cây quá đúng với tập train mà không khái quát được dữ liệu.

In [33]:
clf = RandomForestClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred, labels=np.unique(y_test)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_test, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor              4315               909            31
True Standard          1174              8211           695
True Good                15               871          2442
              precision    recall  f1-score   support

        Poor       0.78      0.82      0.80      5255
    Standard       0.82      0.81      0.82     10080
        Good       0.77      0.73      0.75      3328

    accuracy                           0.80     18663
   macro avg       0.79      0.79      0.79     18663
weighted avg       0.80      0.80      0.80     18663



### Ưu điểm:
Kết quả được cải thiện rõ rệt sau khi phân tích xử lí dữ liệu và lựa chọn thuật toán phù hợp hơn với dữ liệu phức tap
### Nhược điểm
Số lượng feature vẫn rất nhiều khiến mô hình lớn và phức tạp